In [1]:
# https://www.geeksforgeeks.org/scrape-google-search-results-using-python-beautifulsoup/
# https://practicaldatascience.co.uk/data-science/how-to-scrape-google-search-results-using-python

In [ ]:
# !pip install requests_html

In [88]:
import requests
import json
import bs4
from bs4 import BeautifulSoup
import urllib
import requests
import urllib
import pandas as pd
from requests_html import HTML
from requests_html import HTMLSession

In [16]:
def create_search_url(search, url='https://google.com/search?q='):
    search_encoded = urllib.parse.quote_plus(search)
    full_url = url + search_encoded

    return full_url

In [5]:
def get_source(url):
    """Return the source code for the provided URL. 

    Args: 
        url (string): URL of the page to scrape.

    Returns:
        response (object): HTTP response object from requests_html. 
    """


    try:
        session = HTMLSession()
        response = session.get(url)
        return response

    except requests.exceptions.RequestException as e:
        print(e)

In [214]:
def scrape_page_urls(url):
    response = get_source(url)
    links = list(response.html.absolute_links)

    # clean google urls
    google_domains = ('https://www.google.', 
                      'https://google.', 
                      'https://webcache.googleusercontent.', 
                      'http://webcache.googleusercontent.', 
                      'https://policies.google.',
                      'https://support.google.',
                      'https://maps.google.')

    for url in links[:]:
        if url.startswith(google_domains):
            links.remove(url)

    return links

In [12]:
full_url = create_search_url("gluten free baking recipes")
response = get_source(full_url)
search_urls = scrape_page_urls(response)
print(search_urls)

['https://www.foodandwine.com/cooking-techniques/baking/fw-editor-picks-19-best-gluten-free-baking-recipes', 'https://www.tasteofhome.com/recipes/gluten-free-snowballs/', 'https://www.tasteofhome.com/collection/gluten-free-baking-recipes/', 'https://heartbeetkitchen.com/gluten-free-dessert-recipes/', 'https://glutenfreebaking.com/recipes/', 'https://thetoastedpinenut.com/gluten-free-dessert-recipes/#breads', 'https://www.foodnetwork.com/recipes/photos/gluten-free-baking', 'https://www.bonappetit.com/recipes/desserts/slideshow/gluten-free-desserts-recipes', 'https://www.bbcgoodfood.com/recipes/collection/gluten-free-baking-recipes', 'https://www.tasteofhome.com/recipes/gluten-free-peanut-butter-blondies/', 'https://www.simplyrecipes.com/a-beginner-s-guide-to-gluten-free-baking-5202979', 'https://thetoastedpinenut.com/gluten-free-dessert-recipes/#bars', 'https://thetoastedpinenut.com/gluten-free-dessert-recipes/#cookies', 'https://thetoastedpinenut.com/gluten-free-dessert-recipes/']


In [215]:
# response = get_source('https://glutenfreeonashoestring.com/gluten-free-recipes/')
page_urls = scrape_page_urls('https://glutenfreeonashoestring.com/gluten-free-recipes/')
print(page_urls[:5])

['https://glutenfreeonashoestring.com/gluten-free-red-velvet-chocolate-chip-cookies/', 'https://glutenfreeonashoestring.com/about/privacy-policy/', 'https://glutenfreeonashoestring.com/no-bake-gluten-free-bars/', 'https://glutenfreeonashoestring.com/gluten-free-cinnamon-rolls/', 'https://glutenfreeonashoestring.com/brown-bread/']


In [151]:
def get_gf_on_a_shoestring_links(url):
    response = get_source('https://glutenfreeonashoestring.com/gluten-free-recipes/')
    page_urls = scrape_page_urls(response)
    print(f"\nRetrieved {len(page_urls)} recipe links\n")

    return page_urls

In [206]:
def get_recipe_info(url):
    recipe_dict = {}

    html = requests.get(url)
    soup = BeautifulSoup(html.text, 'lxml')

    try:
        scripts = [item for item in soup.find("script", {"class": "yoast-schema-graph"})]
        scripts_json = json.loads(scripts[0])

        for nbr in range(0, len(scripts_json["@graph"])):
            if "recipeIngredient" in list(scripts_json["@graph"][nbr].keys()):
                recipe_info = scripts_json["@graph"][nbr]
                # TODO: use title instead of url
                recipe_dict[recipe_info["name"]] = recipe_info

                return recipe_dict
    except:
        recipe_dict.update(url="")

In [207]:
recipe_info = {}

for url in shoestring_urls:
    recipe_info[url] = get_recipe_info(url)

## WORKING STUFF

In [82]:
html = requests.get('https://glutenfreeonashoestring.com/gluten-free-red-velvet-chocolate-chip-cookies/')
soup = BeautifulSoup(html.text, 'lxml')

In [112]:
scripts = [item for item in soup.find("script", {"class": "yoast-schema-graph"})]

In [113]:
scripts_json = json.loads(scripts[0])

In [205]:
scripts_json["@graph"][5]["name"]

'Gluten Free Red Velvet Chocolate Chip Cookies | Soft &amp; Chewy'

In [153]:
shoestring_urls = get_gf_on_a_shoestring_links('https://glutenfreeonashoestring.com/gluten-free-recipes/')

Retrieved 43 recipe links


In [216]:
pwd

'/home/ec2-user/SageMaker/just-a-recipe-generator'

In [226]:
run src/data/scrape_glutenfreeonashoestring.py


Retrieved 43 recipe links


Retrieved 37 links



## Scrape all recipes

Need to use selenium or come up with different urls